In [1]:
import pandas as pd
import re
from collections import Counter

from tqdm import tqdm, tqdm_pandas
from tqdm._tqdm_notebook import tqdm_notebook

In [2]:
import sys
sys.path.insert(0, '/home/ALPHA/shtanko-la/Programming/preprocessing/')
sys.path.insert(0, '/home/ALPHA/shtanko-la/Programming/')
from prod.ConfigParser import ConfigParser
from routines import file_routines

In [4]:
data = pd.read_excel('../../tech_support/tech_november.xlsx')

In [5]:
data_corp = data[data.EVENT_OWNER == 'CORPORATE']

In [18]:
preprocess_util = ConfigParser(config_file='../config.json')

In [6]:
data_corp.CHAT_ID.unique().shape

(22066,)

In [7]:
data_corp_short = data_corp.loc[:50000]
#data_corp_short = data_corp

In [19]:
tqdm_notebook.pandas(tqdm())
data_corp_short['proc'] = data_corp_short.progress_apply(lambda x: preprocess_util.filter_string(str(x['EVENT_MESSAGE'])), axis=1)


0it [00:00, ?it/s]


A Jupyter Widget

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
pp = data_corp_short.proc.tolist()

In [ ]:
import json
with open('../../preprocessing/dicts/actions_dict.json', 'r') as f:
    act_d = json.load(fp=f)
    
with open('../../preprocessing/dicts/objects_dict.json', 'r') as f:
    obj_d = json.load(fp=f)    

In [ ]:
ma = {}
for k in obj_d:
    ma[k] = 'OBJECT'
for k in act_d:
    ma[k] = 'ACTION'

In [ ]:
pp_filtered = [([ma.get(i, None) for i in p.split()]) for p in pp]

In [ ]:
data_corp_short.iloc[:40].EVENT_MESSAGE

In [21]:
pp

[' PERSON подавать APPLY в OLD_DESIGN под руководство специалист SUPPORT так HOW в NEW_DESIGN выскакивать ERROR ',
 ' TIME_PERIOD в OLD_DESIGN STATUS - ACCEPT_STAGE',
 'при это I_HAVE_A_PROB TIME_PERIOD NUMBER проходить PAYMENT на NUMBER рубль на заправка, а вечер PAYMENT по CARD в размер NUMBER NOT проходить.',
 'хм...',
 'ладно',
 'прекрасно...',
 ' PERSON уже он DO за помощь',
 ' HOW же тогда проходить PAYMENT на заправка?',
 'ну да',
 ' CAN присылать фото чека',
 'в сутки NUMBER 000 NUMBER',
 'в TIME_PERIOD NUMBER 000 NUMBER',
 'в TIME_PERIOD NUMBER 000 NUMBER - PERSON же уже написать',
 ' TIME_PERIOD PERSON DO LIMIT на весь операция NUMBER 000 NUMBER без уточнение',
 'да, все. под чуткий руководство оператор из SUPPORT ',
 ' NUMBER NUMBER 000 NUMBER',
 ' NUMBER NUMBER 000 NUMBER',
 ' NUMBER NUMBER 000 NUMBER',
 ' NUMBER NUMBER',
 ' PERSON TRANSFER 2 APPLY - с он тоже?',
 'вот что WRITE об операция    блокировка, retail, rus sankt-peterbu retail rus sankt-peterbu neste azs NUMBER',

In [ ]:
pp_filtered_tuples = [[(i,j) for i,j in 
                       zip(zip(p[:-1], p[1:]), zip(p2[:-1], p2[1:])) if i in 
                       [('OBJECT', 'ACTION'), ('ACTION', 'OBJECT')]]
                      for p, p2 in zip(pp_filtered, [piy.split() for piy in pp])]

In [ ]:
pp_ff = [p for p in pp_filtered_tuples if len(p)!=0]

In [ ]:
from itertools import chain
pp_ff2 = [p[1] for p in chain(*pp_ff)]

In [ ]:
Counter(pp_ff2).most_common()

In [ ]:
def ngram_list_from_string(s, n_gram=2):
    ret_list = []
    spl = s.split()
    slices = [slice(i, len(spl) - n_gram + i) for i in range(n_gram)]
    sliced_spl = [spl[sl] for sl in slices]
    for words in zip(*sliced_spl):
        ret_list.append(" ".join(words))
    return ret_list

def ngrams_from_list_of_strings(l, n_gram=2):
    from itertools import chain
    ngrams = []
    for line in l:
        ngrams.append(ngram_list_from_string(line, n_gram=n_gram))
    return list(chain(*ngrams))

In [ ]:
gram1 = ngrams_from_list_of_strings(data_corp_short.proc.tolist(), n_gram=1)
gram2 = ngrams_from_list_of_strings(data_corp_short.proc.tolist())
gram3 = ngrams_from_list_of_strings(data_corp_short.proc.tolist(), n_gram=3)
gram4 = ngrams_from_list_of_strings(data_corp_short.proc.tolist(), n_gram=4)

In [ ]:
grams = [ngrams_from_list_of_strings(data_corp_short.proc.tolist(), n_gram=i) for i in range(5,15)]

In [ ]:
len(set(gram2))

In [ ]:
cnt1 = Counter(gram1)
cnt2 = Counter(gram2)
cnt3 = Counter(gram3)
cnt4 = Counter(gram4)

In [ ]:
cnts = [Counter(gr) for gr in grams]

In [ ]:
cnt2.most_common()

In [ ]:
cnt1.most_common()

In [ ]:
import pymystem3
import pymorphy2

In [ ]:
st = pymystem3.Mystem()

In [ ]:
st.analyze('свой мой твой')

In [ ]:
an = pymorphy2.MorphAnalyzer()

In [ ]:
an.parse('свой')

In [ ]:
act_d2 = {k: {'synonyms': v, 'parents': ['ACTION']} for k,v in act_d.items()}

In [ ]:
act_d3 = {'tokens': act_d2, 'type': 'ACTION' }

In [ ]:
with open('../dicts_beta/actions_dict.json', 'w') as f:
    json.dump(act_d3, fp=f, ensure_ascii=False)

In [ ]:
obj_d2 = {k: {'synonyms': v, 'parents': ['OBJECT']} for k,v in obj_d.items()}

In [ ]:
obj_d3 = {'tokens': obj_d2, 'type': 'OBJECT' }

In [ ]:
with open('../dicts_graph/objects_dict.json', 'w') as f:
    json.dump(obj_d3, fp=f, ensure_ascii=False)